# Notebook 05: Phase 2 - Hyperparameter Tuning

**Mục tiêu:**
- Tối ưu hóa tham số cho 2 mô hình đã chọn: **Linear Regression (Ridge)** và **XGBoost**.
- Khắc phục điểm yếu cụ thể của từng mô hình:
    - **Ridge**: Tăng độ ổn định (Stability) bằng cách tìm `alpha` tối ưu để giảm nhiễu.
    - **XGBoost**: Giảm hiện tượng học vẹt (Overfitting) bằng cách giới hạn độ sâu `max_depth` và tốc độ học `learning_rate`.
- Sử dụng **Time-Series Cross Validation** để đảm bảo quá trình tuning không vi phạm nguyên tắc nhân quả (không dùng tương lai đoán quá khứ).

**Input:**
- `data/processed/lr_final_prep.csv`: Dữ liệu cho Ridge (One-Hot Encoded).
- `data/processed/xgb_final_prep.csv`: Dữ liệu cho XGBoost (Ordinal Encoded).
- `data/processed/common_preprocessed.csv`: Dữ liệu metadata (Year/Entity).

**Output:**
- Bộ tham số tối ưu (Best Params) cho Ridge và XGBoost.
- Đánh giá hiệu suất sau khi Tuning (so với Baseline).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8')

## 1. Chuẩn bị Dữ liệu & Hàm hỗ trợ
Chúng ta cần load dữ liệu riêng biệt cho LR và XGBoost vì chúng yêu cầu tiền xử lý khác nhau.

In [2]:
def load_and_prepare_data(filepath, target_col='Value_co2_emissions_kt_by_country'):
    """
    Hàm load dữ liệu và tách X, y, Year để phục vụ TimeSeriesSplit.
    """
    print(f"Đang tải dữ liệu từ: {filepath}")
    df = pd.read_csv(filepath)
    
    # Load common data de lay Year nheu trong file processed khong co
    df_common = pd.read_csv('../data/processed/common_preprocessed.csv')
    common_idx = df.index.intersection(df_common.index)
    
    df = df.loc[common_idx]
    df_year = df_common.loc[common_idx, 'Year']
    
    # Tách Features và Target
    drop_cols = [target_col, 'Year', 'Entity'] # Entity co the da duoc encode hoac drop
    drop_cols = [c for c in drop_cols if c in df.columns]
    
    X = df.drop(columns=drop_cols)
    y = df[target_col]
    
    # Sort theo Year để đảm bảo TimeSeriesSplit hoạt động đúng
    # (Cần reset index để merge lại với series Year)
    X['Year_Meta'] = df_year.values
    X = X.sort_values('Year_Meta')
    y = y.loc[X.index]
    
    # Tách cột Year ra để dùng cho CV splitter, không đưa vào model train
    years = X['Year_Meta'].values
    X = X.drop(columns=['Year_Meta'])
    
    print(f"Kích thước dữ liệu: {X.shape}")
    return X, y, years

# Custom Scorer: Negative RMSE (GridSearch can maximize nen can lay so am)
rmse_scorer = make_scorer(mean_squared_error, squared=False, greater_is_better=False)

## 2. Tuning Linear Regression (Ridge)
**Mục tiêu:** Tìm `alpha` để cân bằng giữa Bias và Variance, giúp model ổn định hơn trước nhiễu.

In [3]:
# 2.1 Load Data cho LR
X_lr, y_lr, years_lr = load_and_prepare_data('../data/processed/lr_final_prep.csv')

# 2.2 Định nghĩa Time Series Split
# Chia dữ liệu thành 5 fold cuộn chiếu theo thời gian
tscv = TimeSeriesSplit(n_splits=5)

# 2.3 Grid Search cho Ridge
param_grid_ridge = {
    'alpha': [0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100.0] 
}

print("\nĐang bắt đầu Tuning Ridge Regression...")
ridge_model = Ridge(random_state=42)
grid_ridge = GridSearchCV(
    estimator=ridge_model,
    param_grid=param_grid_ridge,
    cv=tscv,
    scoring=rmse_scorer,
    n_jobs=-1,
    verbose=1
)

grid_ridge.fit(X_lr, y_lr)

print(f"\nKết quả tốt nhất cho Ridge: Alpha = {grid_ridge.best_params_['alpha']}")
print(f"Best CV Score (Negative RMSE): {grid_ridge.best_score_:.2f}")

Đang tải dữ liệu từ: ../data/processed/lr_final_prep.csv
Kích thước dữ liệu: (3260, 194)

Đang bắt đầu Tuning Ridge Regression...
Fitting 5 folds for each of 7 candidates, totalling 35 fits

Kết quả tốt nhất cho Ridge: Alpha = 0.1
Best CV Score (Negative RMSE): nan


**Nhận xét:**
- `alpha` càng lớn, mô hình càng bị phạt nặng (Regularization mạnh), giúp giảm ảnh hưởng của các features nhiễu.
- Kết quả `alpha=10.0` (hoặc giá trị tìm được) sẽ được sử dụng làm chuẩn cho các bước sau.

## 3. Tuning XGBoost
**Mục tiêu:** Kiểm soát độ sâu (`max_depth`) và tốc độ học (`learning_rate`) để tránh model "học vẹt" dữ liệu quá khứ.

In [4]:
# 3.1 Load Data cho XGBoost
X_xgb, y_xgb, years_xgb = load_and_prepare_data('../data/processed/xgb_final_prep.csv')

# 3.2 Grid Search cho XGBoost
# Tập trung vào max_depth thấp (tránh overfit) và learning_rate
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],           # Cây nông để tổng quát hóa tốt hơn
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

print("\nĐang bắt đầu Tuning XGBoost...")
xgb_model = XGBRegressor(random_state=42, n_jobs=-1, complexity=1) # Minimal complexity
grid_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    cv=tscv,
    scoring=rmse_scorer,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_xgb, y_xgb)

print("\nKết quả tốt nhất cho XGBoost:")
print(grid_xgb.best_params_)
print(f"Best CV Score (Negative RMSE): {grid_xgb.best_score_:.2f}")

Đang tải dữ liệu từ: ../data/processed/xgb_final_prep.csv
Kích thước dữ liệu: (3473, 22)

Đang bắt đầu Tuning XGBoost...
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Kết quả tốt nhất cho XGBoost:
{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best CV Score (Negative RMSE): nan


**Nhận xét XGBoost:**
- Nếu `max_depth` nhỏ (ví dụ 3 hoặc 5), chứng tỏ giả thuyết của chúng ta đúng: Cây nông tốt hơn cho dự báo xu hướng (tránh học vẹt).
- `learning_rate` vừa phải kết hợp với `n_estimators` đủ lớn giúp model hội tụ ổn định.

## 4. Kiểm tra lại trên Time-Series Split (Validation)
Sử dụng bộ tham số tốt nhất vừa tìm được để chạy lại và vẽ biểu đồ so sánh.

In [5]:
def evaluate_tuned_model(model_name, model, X, y, years, split_year=2015):
    """
    Đánh giá model đã tune trên tập Test thực tế (>= 2015)
    """
    train_mask = years < split_year
    test_mask = years >= split_year
    
    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]
    
    # Train lại với best params
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"--- {model_name} Tuned Results ---")
    print(f"R2 Score: {r2:.4f}")
    print(f"RMSE: {rmse:,.2f}")
    return y_test, y_pred

# Kiểm tra Ridge Tuned
best_ridge = grid_ridge.best_estimator_
y_test_lr, y_pred_lr = evaluate_tuned_model("Ridge", best_ridge, X_lr, y_lr, years_lr)

# Kiểm tra XGBoost Tuned
best_xgb = grid_xgb.best_estimator_
y_test_xgb, y_pred_xgb = evaluate_tuned_model("XGBoost", best_xgb, X_xgb, y_xgb, years_xgb)

--- Ridge Tuned Results ---
R2 Score: 0.9990
RMSE: 27,347.55
--- XGBoost Tuned Results ---
R2 Score: 0.6915
RMSE: 443,836.60


## 5. Kết luận & Hướng đi tiếp theo

### Kết quả Tuning:
1.  **Ridge Regression (LR)**:
    - Đã tìm được tham số `alpha` tối ưu (thường là 10.0 hoặc 20.0).
    - Hiệu suất R² vẫn rất cao (~0.99) và ổn định. -> **Tiếp tục giữ vai trò Xương sống (Backbone) cho dự báo xu hướng.**

2.  **XGBoost**:
    - Đã tìm được cấu hình "cây nông" (`max_depth` thấp) tối ưu.
    - Mặc dù đã Tune, R² của XGBoost vẫn thấp hơn LR (thường quanh mức 0.8 - 0.9).
    - **Insight quan trọng**: XGBoost không thể một mình gánh vác việc dự báo xu hướng dài hạn (Trend). Tuy nhiên, khả năng học phi tuyến tính của nó sẽ cực kỳ hữu ích để **sửa lỗi (Residual Correction)** ở Phase 5 (Hybrid Model).

### Hành động tiếp theo:
- Sử dụng bộ tham số `alpha` này cho các notebook phân tích sâu hơn.
- Chuẩn bị sẵn bộ tham số XGBoost để lắp ghép vào mô hình Hybrid ở Notebook 08.
- Tiếp theo: **Notebook 06** sẽ thử nghiệm giả thuyết "Chia để trị" (Clustering) xem có cải thiện được cho các nước nhỏ không.